In [1]:
import pandas as pd
import numpy as np

In [2]:
commuting_data = pd.read_csv("../data/processed/cleaned_commuting_data.csv")
population_data = pd.read_csv("../data/processed/cleaned_population_data.csv")

In [3]:
print(commuting_data.columns)
print(population_data.columns)

Index(['Geography', 'Car_Truck_Van_Less_Than_15', 'Car_Truck_Van_15_to_29',
       'Car_Truck_Van_30_to_44', 'Car_Truck_Van_45_to_59',
       'Car_Truck_Van_More_Than_60', 'Car_Truck_Van_Avg_Duration',
       'Public_Transit_Less_Than_15', 'Public_Transit_15_to_29',
       'Public_Transit_30_to_44', 'Public_Transit_45_to_59',
       'Public_Transit_More_Than_60', 'Public_Transit_Avg_Duration',
       'Active_Transportation_Less_Than_15', 'Active_Transportation_15_to_29',
       'Active_Transportation_30_to_44', 'Active_Transportation_45_to_59',
       'Active_Transportation_More_Than_60',
       'Active_Transportation_Avg_Duration',
       'Motorcycle_Scooter_Moped_Less_Than_15',
       'Motorcycle_Scooter_Moped_15_to_29',
       'Motorcycle_Scooter_Moped_30_to_44',
       'Motorcycle_Scooter_Moped_45_to_59',
       'Motorcycle_Scooter_Moped_More_Than_60',
       'Motorcycle_Scooter_Moped_Avg_Duration', 'Other_Less_Than_15',
       'Other_15_to_29', 'Other_30_to_44', 'Other_45_to_59',


In [4]:
# Use regex to remove the trailing "ixxxx" from the Geography column of the commuting_data.csv table
commuting_data["Geography"] = commuting_data["Geography"].str.replace(r"\s*i\d+$", "", regex=True)

# Verify the cleaned Geography column
print(commuting_data["Geography"].head())

0             Division No.  1
1    Division No.  1, Subd. V
2         Portugal Cove South
3                   Trepassey
4                 St. Shott's
Name: Geography, dtype: object


In [5]:
# Merge datasets after standardizing the Geography
combined_data = pd.merge(commuting_data, population_data, on="Geography", how="inner")

Let's sort the merged data alphabetically by the "Geography" and then print out a sample of the data.

In [6]:
combined_data = combined_data.sort_values(by="Geography")
combined_data = combined_data.reset_index(drop=True)

In [7]:
print(combined_data.head())
print(combined_data.info())

         Geography  Car_Truck_Van_Less_Than_15  Car_Truck_Van_15_to_29  \
0  105 Mile Post 2                         0.0                     0.0   
1            Abbey                         0.0                     0.0   
2       Abbotsford                     17450.0                 14690.0   
3         Abercorn                        40.0                     0.0   
4         Aberdeen                        45.0                    85.0   

   Car_Truck_Van_30_to_44  Car_Truck_Van_45_to_59  Car_Truck_Van_More_Than_60  \
0                     0.0                     0.0                         0.0   
1                     0.0                     0.0                         0.0   
2                  7015.0                  2275.0                      1135.0   
3                    15.0                     0.0                         0.0   
4                    30.0                    15.0                        35.0   

   Car_Truck_Van_Avg_Duration  Public_Transit_Less_Than_15  \
0     

Throughout this merged data table, we have some duplicate values for some geographies. This occurs because some of the geographies from the Statistics Canada dataset are both cities and district municipalities under the same name. Let's check for duplicate entries and let's resolve them before we move onto the data analysis stage. This will ensure we have clean and reliable data.

In [12]:
# Check for duplicates in the merged dataset
duplicates = combined_data[combined_data.duplicated(subset="Geography", keep=False)]

print(f"Number of duplicate entries in merged dataset: {len(duplicates)}")
print(duplicates)

Number of duplicate entries in merged dataset: 542
      Geography  Car_Truck_Van_Less_Than_15  Car_Truck_Van_15_to_29  \
4      Aberdeen                        45.0                    85.0   
5      Aberdeen                        85.0                    80.0   
30     Akulivik                        65.0                     0.0   
31     Akulivik                         0.0                     0.0   
43     Alberton                         0.0                     0.0   
...         ...                         ...                     ...   
4572  Woodstock                      1855.0                   865.0   
4573  Woodstock                      7950.0                  4400.0   
4589   Yarmouth                      3800.0                  2435.0   
4590   Yarmouth                       735.0                   550.0   
4591   Yarmouth                      2410.0                  1585.0   

      Car_Truck_Van_30_to_44  Car_Truck_Van_45_to_59  \
4                       30.0            

We can see that we have  542 rows of data that have a duplicate. To resolve this, we can aggregate data under the same Geography name. For numeric columns, we can take the mean value, whilst for the categorical data, namely "Geography" and "Province or Territory Abbreviation" we can just take the first instance, since they will be the same for both rows.

In [14]:
# Identify non-numerical columns first
non_numerical_cols = ["Geography", "Province or Territory Abbreviation"]

# Seperate numerical and non-numerical columns
numeric_cols = [col for col in combined_data.columns if col not in non_numerical_cols]

# Aggregate numerical columns by Geography
aggregated_numeric = combined_data.groupby("Geography")[numeric_cols].mean().reset_index()

# Aggregate non-numerical columns by Geography
aggregated_non_numeric = combined_data.groupby("Geography")[["Province or Territory Abbreviation"]].first().reset_index()

# Merge the two aggregated datasets
final_aggregated_data = pd.merge(aggregated_numeric, aggregated_non_numeric, on="Geography")

We should double check that the aggreation process worked correctly, and then our data will be ready to export into one larger .csv file for exploratory data analysis.

In [15]:
# Verify that duplicates are resolved
duplicates_check = final_aggregated_data[final_aggregated_data.duplicated(subset="Geography", keep=False)]
print(f"Number of duplicate geographies after cleaning: {len(duplicates_check)}")

# Save the cleaned and aggregated data
final_aggregated_data.to_csv("../data/processed/cleaned_combined_data.csv", index=False)

Number of duplicate geographies after cleaning: 0
